In [1]:
%reload_ext autoreload
%autoreload 2

import os
from pathlib import Path
import multiprocessing

import jax
import jax.numpy as jnp
import pandas as pd
import numpy as np
import numpyro
import numpyro.distributions as dist

from hbmep.config import Config
from hbmep.model import Baseline
from hbmep.model.utils import Site as site
from hbmep.utils.constants import RECTIFIED_LOGISTIC

PLATFORM = "cpu"
jax.config.update("jax_platforms", PLATFORM)
numpyro.set_platform(PLATFORM)

cpu_count = multiprocessing.cpu_count() - 2
numpyro.set_host_device_count(cpu_count)
numpyro.enable_x64()
numpyro.enable_validation()


In [2]:
root_path = Path(os.getcwd()).parent.parent.parent.parent.absolute()
toml_path = os.path.join(root_path, "configs/rcml/biceps.toml")

config = Config(toml_path=toml_path)


2023-08-09 12:58:16,589 - hbmep.config - INFO - Verifying configuration ...
2023-08-09 12:58:16,589 - hbmep.config - INFO - Success!


#### Load data and preprocess

In [3]:
class RectifiedLogistic(Baseline):
    LINK = RECTIFIED_LOGISTIC

    def __init__(self, config: Config):
        super(RectifiedLogistic, self).__init__(config=config)

    def _model(self, subject, features, intensity, response_obs=None):
        intensity = intensity.reshape(-1, 1)
        intensity = np.tile(intensity, (1, self.n_response))

        feature0 = features[0, ...].reshape(-1,)

        n_data = intensity.shape[0]
        n_subject = np.unique(subject).shape[0]
        n_feature0 = np.unique(feature0).shape[0]

        with numpyro.plate(site.n_response, self.n_response, dim=-1):
            with numpyro.plate(site.n_subject, n_subject, dim=-2):
                """ Hyper-priors """
                mu_a = numpyro.sample(
                    site.mu_a,
                    dist.TruncatedNormal(150, 50, low=0)
                )
                sigma_a = numpyro.sample(site.sigma_a, dist.HalfNormal(50))

                sigma_b = numpyro.sample(site.sigma_b, dist.HalfNormal(0.1))

                sigma_L = numpyro.sample(site.sigma_L, dist.HalfNormal(0.05))
                sigma_H = numpyro.sample(site.sigma_H, dist.HalfNormal(5))
                sigma_v = numpyro.sample(site.sigma_v, dist.HalfNormal(10))

                with numpyro.plate("n_feature0", n_feature0, dim=-3):
                    """ Priors """
                    a = numpyro.sample(
                        site.a,
                        dist.TruncatedNormal(mu_a, sigma_a, low=0)
                    )
                    b = numpyro.sample(site.b, dist.HalfNormal(sigma_b))

                    L = numpyro.sample(site.L, dist.HalfNormal(sigma_L))
                    H = numpyro.sample(site.H, dist.HalfNormal(sigma_H))
                    v = numpyro.sample(site.v, dist.HalfNormal(sigma_v))

                    g_1 = numpyro.sample(site.g_1, dist.Exponential(0.01))
                    g_2 = numpyro.sample(site.g_2, dist.Exponential(0.01))

        """ Model """
        mu = numpyro.deterministic(
            site.mu,
            L[feature0, subject]
            + jnp.maximum(
                0,
                -1
                + (H[feature0, subject] + 1)
                / jnp.power(
                    1
                    + (jnp.power(1 + H[feature0, subject], v[feature0, subject]) - 1)
                    * jnp.exp(-b[feature0, subject] * (intensity - a[feature0, subject])),
                    1 / v[feature0, subject]
                )
            )
        )
        beta = numpyro.deterministic(
            site.beta,
            g_1[feature0, subject] + g_2[feature0, subject] * (1 / mu)
        )

        """ Observation """
        with numpyro.plate(site.data, n_data):
            return numpyro.sample(
                site.obs,
                dist.Gamma(concentration=mu * beta, rate=beta).to_event(1),
                obs=response_obs
            )


model = RectifiedLogistic(config=config)


2023-08-09 12:58:16,623 - hbmep.model.baseline - INFO - Initialized model with rectified_logistic link


In [4]:
df = pd.read_csv(model.csv_path)

ind = df.pulse_amplitude.isin([0])
df = df[~ind].reset_index(drop=True).copy()

subset = ['-C5L', '-C6L', '-C7L', '-C8L']
ind = df.compound_position.isin(subset)
df = df[ind].reset_index(drop=True).copy()

# df[model.features] = \
#     df.compound_position \
#         .apply(lambda x: x.split("-")) \
#         .apply(lambda x: (x[0][:2] + "-" + x[1][:2], x[0][-1])) \
#         .apply(pd.Series)

""" Process """
df, encoder_dict = model.load(df=df)

""" Plot """
model.plot(df=df, encoder_dict=encoder_dict)

2023-08-09 12:58:16,661 - hbmep.dataset.core - INFO - Artefacts will be stored here - /home/vishu/repos/hbmep-paper/reports/J_RCML_000/biceps
2023-08-09 12:58:16,662 - hbmep.dataset.core - INFO - Copied config to /home/vishu/repos/hbmep-paper/reports/J_RCML_000/biceps
2023-08-09 12:58:16,663 - hbmep.dataset.core - INFO - Processing data ...
2023-08-09 12:58:16,664 - hbmep.utils.utils - INFO - func:load took: 0.00 sec
2023-08-09 12:58:16,666 - hbmep.dataset.core - INFO - Plotting dataset ...


2023-08-09 12:58:22,576 - hbmep.dataset.core - INFO - Saved to /home/vishu/repos/hbmep-paper/reports/J_RCML_000/biceps/dataset.pdf
2023-08-09 12:58:22,576 - hbmep.utils.utils - INFO - func:plot took: 5.91 sec


In [5]:
df.shape

(1604, 44)

In [7]:
mcmc, posterior_samples = model.run_inference(df=df)


2023-08-09 12:58:36,149 - hbmep.model.baseline - INFO - Running inference with rectified_logistic ...


  0%|          | 0/10000 [00:00<?, ?it/s]

  0%|          | 0/10000 [00:00<?, ?it/s]

  0%|          | 0/10000 [00:00<?, ?it/s]

  0%|          | 0/10000 [00:00<?, ?it/s]

2023-08-09 13:30:22,416 - hbmep.utils.utils - INFO - func:run_inference took: 31 min and 46.27 sec


In [8]:
mcmc.print_summary(prob=.95)



                mean       std    median      2.5%     97.5%     n_eff     r_hat
  H[0,0,0]      3.30      1.39      2.95      1.55      5.89   8835.76      1.00
  H[0,1,0]      4.70      0.50      4.62      3.87      5.72   8325.09      1.00
  H[0,2,0]      3.04      0.06      3.03      2.92      3.16  21070.21      1.00
  H[0,3,0]      5.53      1.94      5.06      3.05      9.14   4379.40      1.00
  H[0,4,0]      2.73      0.21      2.69      2.38      3.16   5135.56      1.00
  H[0,5,0]     13.16      5.24     11.92      5.82     23.70  12987.01      1.00
  H[0,6,0]      4.28      0.67      4.17      3.15      5.59  11568.25      1.00
  H[0,7,0]      2.26      1.60      1.77      0.83      5.16   8726.68      1.00
  H[1,0,0]      1.11      0.15      1.09      0.86      1.41   8384.04      1.00
  H[1,1,0]      6.19      4.25      4.98      1.37     14.70  14598.60      1.00
  H[1,2,0]      1.99      0.09      1.98      1.81      2.17   8819.18      1.00
  H[1,3,0]      9.32      4

In [9]:
model.render_recruitment_curves(df=df, encoder_dict=encoder_dict, posterior_samples=posterior_samples)



2023-08-09 13:30:23,308 - hbmep.model.baseline - INFO - Generating predictions ...


2023-08-09 13:31:02,619 - hbmep.utils.utils - INFO - func:predict took: 39.30 sec
2023-08-09 13:31:02,853 - hbmep.model.baseline - INFO - Rendering recruitment curves ...
2023-08-09 13:31:19,011 - hbmep.model.baseline - INFO - Saved to /home/vishu/repos/hbmep-paper/reports/J_RCML_000/biceps/recruitment_curves.pdf
2023-08-09 13:31:19,012 - hbmep.utils.utils - INFO - func:render_recruitment_curves took: 55.70 sec


In [10]:
model.render_predictive_check(df=df, encoder_dict=encoder_dict, posterior_samples=posterior_samples)


2023-08-09 13:31:19,048 - hbmep.model.baseline - INFO - Generating predictions ...


2023-08-09 13:32:02,977 - hbmep.utils.utils - INFO - func:predict took: 43.92 sec
2023-08-09 13:32:03,432 - hbmep.model.baseline - INFO - Rendering Posterior Predictive Check ...
2023-08-09 13:32:23,718 - hbmep.model.baseline - INFO - Saved to /home/vishu/repos/hbmep-paper/reports/J_RCML_000/biceps/posterior_predictive_check.pdf
2023-08-09 13:32:23,720 - hbmep.utils.utils - INFO - func:render_predictive_check took: 1 min and 4.67 sec


In [11]:
model.save(mcmc=mcmc)

2023-08-09 13:32:23,860 - hbmep.model.baseline - INFO - Saving inference data ...


2023-08-09 13:32:49,869 - hbmep.model.baseline - INFO - Saved to /home/vishu/repos/hbmep-paper/reports/J_RCML_000/biceps/mcmc.nc
2023-08-09 13:32:49,870 - hbmep.model.baseline - INFO - Rendering convergence diagnostics ...
2023-08-09 13:33:24,225 - hbmep.model.baseline - INFO - Saved to /home/vishu/repos/hbmep-paper/reports/J_RCML_000/biceps/diagnostics.csv
2023-08-09 13:33:24,226 - hbmep.model.baseline - INFO - Evaluating model ...
/home/vishu/repos/hbmep-paper/.venv/lib/python3.11/site-packages/arviz/stats/stats.py:803: UserWarning: Estimated shape parameter of Pareto distribution is greater than 0.7 for one or more samples. You should consider using a more robust model, this is because importance sampling is less likely to work well if the marginal posterior and LOO posterior are very different. This is more likely to happen with a non-robust model and highly influential observations.
  warnings.warn(
2023-08-09 13:33:29,532 - hbmep.model.baseline - INFO - ELPD LOO (Log): 1432.28
/h